In [ ]:
!pip install langchain langchain-community langchain-openai torch torchvision torchaudio nougat-ocr git+https://github.com/facebookresearch/nougat

In [ ]:
!pip install git+https://github.com/VikParuchuri/marker faiss-cpu

In [ ]:
!pip install transformers==4.38.2

In [ ]:
import os
from marker.models import load_all_models
from marker.convert import convert_single_pdf
from marker.output import save_markdown
from marker.logger import configure_logging
from dotenv import load_dotenv
import subprocess
import torch

load_dotenv()
configure_logging()

In [ ]:
def generate_nougat_markdown(pdf_file_path, output_folder_path):
    file_name = os.path.basename(pdf_file_path).split('.')[0]
    if not os.path.exists(f'/kaggle/input/results/{file_name}/{file_name}.mmd'):
        cli_command = ["nougat", "--markdown", "pdf", pdf_file_path, "--out", output_folder_path]
        result = subprocess.run(cli_command, stdout = subprocess.PIPE, stderr = subprocess.PIPE, universal_newlines = True, encoding='utf-8')
        print(result.stdout)
        print(result.stderr)

def generate_marker_markdown(file_path_pdf, output_path):
    file_name = os.path.basename(file_path_pdf).split('.')[0]
    if not os.path.exists(f'/kaggle/input/results/{file_name}/{file_name}.md'):
        model_lst = load_all_models()
        full_text, images, out_meta = convert_single_pdf(file_path_pdf, model_lst)
        fname = os.path.basename(file_path_pdf)
        subfolder_path = save_markdown(output_path, fname, full_text, images, out_meta)
        print(f"Saved markdown to the {subfolder_path} folder")
        return subfolder_path
    else:
        return f'/kaggle/input/results/{file_name}'

In [ ]:
import glob
pdfs = glob.glob('/kaggle/input/eval-files/*.pdf')
print(pdfs)
for file_path_pdf in pdfs:
    output_folder_name = os.path.basename(file_path_pdf).split('.')[0]
    output_path_pdf = f'/kaggle/working/{output_folder_name}'
    output_path = '/kaggle/working/'
    subfolder_path = generate_marker_markdown(file_path_pdf, output_path)
    generate_nougat_markdown(file_path_pdf, output_path_pdf)